 # 学习曲线 (Learning Curves)

---

## 📚 学习目标

通过本教程，你将学会：
- 理解学习曲线的概念和作用
- 通过学习曲线诊断过拟合和欠拟合
- 学会使用Scikit-learn绘制学习曲线
- 掌握模型调优的策略

## 📋 前置知识

- 偏差-方差权衡
- 线性回归和多项式回归
- 训练集和验证集概念

## ⏱️ 预计时间

25-35分钟

---

## 📖 第1部分：理论背景

### 什么是学习曲线？

**学习曲线**是展示模型在不同训练集大小下的训练误差和验证误差变化趋势的图表。它帮助我们：

1. 诊断模型是否过拟合或欠拟合
2. 判断是否需要更多数据
3. 决定模型复杂度是否合适

### 学习曲线解读

| 模式 | 特征 | 诊断 | 解决方案 |
|------|------|------|----------|
| 欠拟合 | 训练误差高，两条曲线接近 | 高偏差 | 增加模型复杂度 |
| 过拟合 | 训练误差低，验证误差高 | 高方差 | 增加数据/正则化 |
| 良好拟合 | 两条曲线都低且接近 | 平衡 | 模型合适 |

### 偏差-方差权衡

$$\text{总误差} = \text{偏差}^2 + \text{方差} + \text{噪声}$$

> 💡 **关键洞察**: 学习曲线是诊断模型问题的最有效工具之一。

---

## 💻 第2部分：代码实现

### 步骤1: 导入库和配置

In [ ]:
# ============================================================
# 导入必要的库
# ============================================================

import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.metrics import mean_squared_error

# ============================================================
# 配置
# ============================================================

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

print("✓ 配置完成")

### 步骤2: 生成数据

In [ ]:
# ============================================================
# 生成二次关系数据
# ============================================================

n_samples = 100

# 生成特征
X = 6 * np.random.rand(n_samples, 1) - 3

# 生成目标：y = 0.5x² + x + 2 + 噪声
y = 0.5 * X**2 + X + 2 + np.random.randn(n_samples, 1)

print(f"✓ 生成 {n_samples} 个样本")
print(f"  X范围: [{X.min():.2f}, {X.max():.2f}]")

### 步骤3: 手动实现学习曲线

In [ ]:
# ============================================================
# 手动实现学习曲线函数
# ============================================================

def plot_learning_curves_manual(model, X, y, title='学习曲线'):
    """
    手动绘制学习曲线
    
    参数:
        model: 机器学习模型
        X: 特征矩阵
        y: 目标向量
        title: 图表标题
    """
    # 划分训练集和验证集
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, random_state=RANDOM_SEED
    )
    
    train_errors = []
    val_errors = []
    
    # 对不同大小的训练集计算误差
    for m in range(1, len(X_train)):
        # 使用前m个样本训练
        model.fit(X_train[:m], y_train[:m])
        
        # 计算训练误差和验证误差
        y_train_pred = model.predict(X_train[:m])
        y_val_pred = model.predict(X_val)
        
        train_errors.append(mean_squared_error(y_train[:m], y_train_pred))
        val_errors.append(mean_squared_error(y_val, y_val_pred))
    
    # 绘图
    plt.plot(np.sqrt(train_errors), 'r-', linewidth=2, label='训练误差')
    plt.plot(np.sqrt(val_errors), 'b-', linewidth=2, label='验证误差')
    plt.xlabel('训练样本数')
    plt.ylabel('RMSE')
    plt.title(title, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 4)

print("✓ 学习曲线函数定义完成")

---

## 🔬 第3部分：三种拟合状态对比

### 1. 欠拟合（线性模型 → 二次数据）

In [ ]:
# ============================================================
# 欠拟合示例：线性回归拟合二次数据
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. 欠拟合：线性模型
plt.subplot(1, 3, 1)
lin_reg = LinearRegression()
plot_learning_curves_manual(lin_reg, X, y, '欠拟合 (线性模型)')

# 2. 良好拟合：二次模型
plt.subplot(1, 3, 2)
poly2_model = Pipeline([
    ('poly', PolynomialFeatures(degree=2, include_bias=False)),
    ('lin_reg', LinearRegression())
])
plot_learning_curves_manual(poly2_model, X, y, '良好拟合 (2次多项式)')

# 3. 过拟合：高次模型
plt.subplot(1, 3, 3)
poly10_model = Pipeline([
    ('poly', PolynomialFeatures(degree=10, include_bias=False)),
    ('lin_reg', LinearRegression())
])
plot_learning_curves_manual(poly10_model, X, y, '过拟合 (10次多项式)')

plt.tight_layout()
plt.show()

print("\n学习曲线解读:")
print("  欠拟合: 两条曲线都高且接近 → 模型太简单")
print("  良好拟合: 两条曲线都低且接近 → 模型复杂度合适")
print("  过拟合: 训练误差低，验证误差高 → 模型太复杂")

---

## 📊 第4部分：使用Scikit-learn的learning_curve

In [ ]:
# ============================================================
# 使用sklearn的learning_curve函数
# ============================================================

def plot_sklearn_learning_curve(estimator, X, y, title='学习曲线', cv=5):
    """
    使用sklearn的learning_curve绘制学习曲线
    
    参数:
        estimator: 估计器
        X: 特征
        y: 目标
        title: 标题
        cv: 交叉验证折数
    """
    # 计算学习曲线
    train_sizes, train_scores, val_scores = learning_curve(
        estimator, X, y.ravel(),
        train_sizes=np.linspace(0.1, 1.0, 10),  # 训练集大小比例
        cv=cv,                                   # 交叉验证折数
        scoring='neg_mean_squared_error',        # 评分标准
        n_jobs=-1                                # 并行计算
    )
    
    # 转换为RMSE
    train_rmse = np.sqrt(-train_scores)
    val_rmse = np.sqrt(-val_scores)
    
    # 计算均值和标准差
    train_mean = train_rmse.mean(axis=1)
    train_std = train_rmse.std(axis=1)
    val_mean = val_rmse.mean(axis=1)
    val_std = val_rmse.std(axis=1)
    
    # 绘图
    plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std,
                     alpha=0.1, color='r')
    plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std,
                     alpha=0.1, color='b')
    plt.plot(train_sizes, train_mean, 'r-', linewidth=2, label='训练误差')
    plt.plot(train_sizes, val_mean, 'b-', linewidth=2, label='验证误差')
    
    plt.xlabel('训练样本数')
    plt.ylabel('RMSE')
    plt.title(title, fontweight='bold')
    plt.legend(loc='best')
    plt.grid(True, alpha=0.3)

print("✓ sklearn学习曲线函数定义完成")

In [ ]:
# ============================================================
# 绘制带置信区间的学习曲线
# ============================================================

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

models = [
    ('欠拟合 (线性)', LinearRegression()),
    ('良好拟合 (2次)', Pipeline([
        ('poly', PolynomialFeatures(degree=2, include_bias=False)),
        ('lin_reg', LinearRegression())
    ])),
    ('过拟合 (10次)', Pipeline([
        ('poly', PolynomialFeatures(degree=10, include_bias=False)),
        ('scaler', StandardScaler()),  # 添加标准化避免数值问题
        ('lin_reg', LinearRegression())
    ]))
]

for i, (name, model) in enumerate(models):
    plt.subplot(1, 3, i+1)
    plot_sklearn_learning_curve(model, X, y, title=name)

plt.tight_layout()
plt.show()

---

## 🔧 第5部分：正则化对学习曲线的影响

In [ ]:
# ============================================================
# 正则化对过拟合的缓解作用
# ============================================================

fig, axes = plt.subplots(1, 4, figsize=(16, 4))

alphas = [0, 0.1, 1, 10]  # 正则化强度

for i, alpha in enumerate(alphas):
    plt.subplot(1, 4, i+1)
    
    if alpha == 0:
        model = Pipeline([
            ('poly', PolynomialFeatures(degree=10, include_bias=False)),
            ('scaler', StandardScaler()),
            ('reg', LinearRegression())
        ])
        title = '无正则化'
    else:
        model = Pipeline([
            ('poly', PolynomialFeatures(degree=10, include_bias=False)),
            ('scaler', StandardScaler()),
            ('reg', Ridge(alpha=alpha))
        ])
        title = f'Ridge (α={alpha})'
    
    plot_sklearn_learning_curve(model, X, y, title=title)

plt.tight_layout()
plt.show()

print("\n观察结论:")
print("  - 无正则化：训练误差很低，但验证误差高（过拟合）")
print("  - 适度正则化：两条曲线更接近（减少过拟合）")
print("  - 过强正则化：两条曲线都升高（可能欠拟合）")

---

## 📈 第6部分：判断是否需要更多数据

In [ ]:
# ============================================================
# 通过学习曲线判断是否需要更多数据
# ============================================================

# 生成更多数据
n_samples_large = 500
X_large = 6 * np.random.rand(n_samples_large, 1) - 3
y_large = 0.5 * X_large**2 + X_large + 2 + np.random.randn(n_samples_large, 1)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 高方差模型（可能受益于更多数据）
high_variance_model = Pipeline([
    ('poly', PolynomialFeatures(degree=10, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lin_reg', LinearRegression())
])

plt.subplot(1, 2, 1)
plot_sklearn_learning_curve(high_variance_model, X, y, '100个样本')
plt.ylim(0, 3)

plt.subplot(1, 2, 2)
plot_sklearn_learning_curve(high_variance_model, X_large, y_large, '500个样本')
plt.ylim(0, 3)

plt.tight_layout()
plt.show()

print("\n判断规则:")
print("  - 如果验证曲线仍在下降：更多数据可能有帮助")
print("  - 如果两条曲线已收敛且差距大：需要改变模型，不是数据")
print("  - 如果两条曲线都高：增加模型复杂度或获取更好的特征")

---

## 📝 总结

### 学习曲线诊断指南

| 现象 | 诊断 | 解决方案 |
|------|------|----------|
| 两条曲线都高且接近 | 高偏差（欠拟合）| 增加模型复杂度、添加特征 |
| 训练误差低，验证误差高 | 高方差（过拟合）| 增加数据、正则化、减少特征 |
| 验证曲线持续下降 | 需要更多数据 | 收集更多训练数据 |
| 两条曲线都低且接近 | 良好拟合 | 模型适合当前任务 |

### 关键要点

1. ✅ 学习曲线是诊断模型问题的重要工具
2. ✅ 关注训练误差和验证误差的**差距**和**趋势**
3. ✅ 高偏差需要更复杂的模型
4. ✅ 高方差需要更多数据或正则化

### 下一步学习

- 📘 验证曲线（调参专用）
- 📙 交叉验证策略
- 📕 超参数调优

## 📚 参考资料

- [Scikit-learn: learning_curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html)
- 《机器学习实战》第4章